#### MVP: 
 * a streamlit dashboard and a tableau dashboard
 * filter by activities
 * **total time spent on activites by week**

#### ideas for this data: 
 * convert the map data to elevation data
 * something with the suffer score
   * suffer per second? suffer per minute? 

#### to do
 * convert start_date column to pandas timestamp 

In [2]:
import pandas as pd
#pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import streamlit as st
import time

import plotly.express as px
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

In [5]:
def get_strava_data(csv_file):
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    activities = pd.read_csv(csv_file)

    while True:
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
        r_df = pd.DataFrame.from_dict(r, orient='columns')     
               
        # if there are no new values (i.e. activities) found in r_df, just break.  otherwise keep going
        if len([v for v in list(r_df['id']) if v not in list(activities['id'])]) == 0:
            activities.drop_duplicates(subset=['id'], inplace=True)
            activities.sort_values(by='id', ascending=False, inplace=True)
            activities.to_csv(csv_file, index=False)
            return activities

        if (not r):   # if no results then exit loop
            break
            
        activities = pd.concat([r_df, activities])
        page += 1
    
    activities.drop_duplicates(subset=['id'], inplace=True)
    activities.sort_values(by='id', ascending=False, inplace=True)
    activities.to_csv(csv_file, index=False)
    return activities

In [4]:
get_strava_data('strava_activities.csv')

0    7465595326
1    7458390918
2    7448404405
3    7431843126
4    7426810587
Name: id, dtype: int64
0    6235816201
1    6231844819
2    6221598053
3    6213215260
4    6209015706
Name: id, dtype: int64


,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,has_kudoed,suffer_score,average_cadence,Unnamed: 0.1,Unnamed: 0,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state
0,2,"{'id': 23871091, 'resource_state': 1}",Evening Ride,21068.8,3634,3843,201.0,Ride,Ride,NaN,...,False,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"{'id': 23871091, 'resource_state': 1}",Morning Run,9791.0,3449,3478,60.0,Run,Run,NaN,...,False,88.0,84.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'id': 23871091, 'resource_state': 1}",Mt. Hood 50k,46716.3,19339,19633,762.0,Run,Run,0.0,...,False,299.0,82.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,"{'id': 23871091, 'resource_state': 1}",Lunch Run,5128.2,1847,1847,32.0,Run,Run,NaN,...,False,22.0,83.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,"{'id': 23871091, 'resource_state': 1}",Lunch Run,6502.7,2164,2172,47.0,Run,Run,NaN,...,False,23.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,2,"{'id': 23871091, 'resource_state': 1}",Bridge Weaves,15454.2,2733,2922,117.9,Ride,Ride,10.0,...,False,NaN,NaN,195.0,NaN,NaN,NaN,NaN,NaN,NaN
798,2,"{'id': 23871091, 'resource_state': 1}",Westside to Tilikum Crossing Loop,12585.1,2040,2040,91.4,Ride,Ride,10.0,...,False,NaN,NaN,196.0,NaN,NaN,NaN,NaN,NaN,NaN
799,2,"{'id': 23871091, 'resource_state': 1}",Maiden Voyage from Forest Park,8532.9,1518,1570,67.7,Ride,Ride,10.0,...,False,NaN,NaN,197.0,NaN,NaN,NaN,NaN,NaN,NaN
800,2,"{'id': 23871091, 'resource_state': 1}",Evening Ride,8836.2,1565,12724,47.3,Ride,Ride,10.0,...,False,NaN,NaN,198.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cols_to_drop = ['Unnamed: 0.1', 'athlete', 'resource_state', 'start_date', 'timezone', 
'utc_offset', 'trainer', 'commute', 'type', 'manual', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'display_hide_heartrate_option',
'upload_id_str', 'external_id', 'from_accepted_tag', 'total_photo_count', 'athlete.resource_state', 'map.id', 'map.summary_polyline', 'map.resource_state', 
'average_watts', 'kilojoules', 'device_watts', 'photo_count', 'heartrate_opt_out', 'upload_id', 'athlete.id', 'workout_type',
'has_heartrate', 'location_city', 'location_state', 'location_country', 'private', 'has_kudoed', 'Unnamed: 0']

strava_activities_raw = pd.read_csv('strava_activities.csv')

strava_activities_clean = strava_activities_raw.drop(labels=cols_to_drop, axis=1)

# meters to miles
strava_activities_clean['distance'] = strava_activities_clean.distance * .000621

# might be a fun metric to plot over time. perhaps my suffering per minute has gone down as ive gotten fitter
strava_activities_clean['SPM'] = strava_activities_clean.suffer_score / (strava_activities_clean.moving_time / 60)

rename = {'start_date_local': 'start_time'}
strava_activities_clean.rename(rename, axis=1, inplace=True)

# convert start_time to pandas Timestamp object. then we can access seconds, minutes, hours, etc. and can add the moving_time
strava_activities_clean['start_time'] = pd.to_datetime(strava_activities_clean['start_time'])

strava_activities_clean.loc[strava_activities_clean.id == 7299973019, 'start_time'] = pd.Timestamp(year=2022, month=6, day=12, hour=2, minute=8, second=36, tz='UTC')

strava_activities_clean.set_index('start_time', inplace=True)

In [130]:
pd.Timestamp(year=2022, month=6, day=12, hour=2, minute=8, second=36, tz='UTC')

Timestamp('2022-06-12 02:08:36+0000', tz='UTC')

In [129]:
strava_activities_clean.index[3]

Timestamp('2022-06-30 00:12:17+0000', tz='UTC')

In [124]:
#strava_activities_clean.index[17] = strava_activities_clean.index[17].replace(day=12)

In [12]:
# my 6/12 run is showing up as 6/13

st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(15)
#st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('M', closed='left').distance.sum().tail(15)

#st.index = st.index.strftime('%B %Y')
st.index = st.index.strftime('Week of %A, %b %d')

In [13]:
#st.index = st.index.strftime('%B %Y')
st.tail(5)

start_time
Week of Monday, Jun 20    21.548141
Week of Monday, Jun 27    36.272175
Week of Monday, Jul 04    31.928715
Week of Monday, Jul 11    36.233611
Week of Monday, Jul 18     6.080211
Name: distance, dtype: float64

In [15]:
#strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].head(30)
strava_activities_clean.head(10)

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,max_speed,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,average_cadence,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-07-14 02:54:33+00:00,Evening Ride,13.083725,3634,3843,201.0,Ride,7465595326,12,1,0,...,16.596,25.0,128.2,167.0,187.4,47.6,4,25.0,NaN,0.412768
2022-07-12 17:51:51+00:00,Morning Run,6.080211,3449,3478,60.0,Run,7458390918,0,2,0,...,3.677,31.0,152.2,162.0,85.2,46.4,0,88.0,84.7,1.530879
2022-07-10 14:00:22+00:00,Mt. Hood 50k,29.010822,19339,19633,762.0,Run,7448404405,8,5,2,...,4.708,20.0,143.9,164.0,1281.4,986.6,6,299.0,82.8,0.927659
2022-07-07 18:59:36+00:00,Lunch Run,3.184612,1847,1847,32.0,Run,7431843126,0,2,0,...,3.581,27.0,140.9,154.0,84.6,64.6,0,22.0,83.5,0.714672
2022-07-06 18:26:08+00:00,Lunch Run,4.038177,2164,2172,47.0,Run,7426810587,1,3,0,...,4.280,24.0,140.4,153.0,84.2,62.8,1,23.0,86.0,0.637708
2022-07-04 16:24:45+00:00,Morning Run,5.492310,3013,3013,54.0,Run,7414762157,0,3,0,...,3.711,24.0,140.3,147.0,57.6,30.8,0,34.0,86.0,0.677066
2022-07-02 22:33:04+00:00,Afternoon Run,10.329403,6001,6001,329.0,Run,7405546458,4,5,0,...,4.196,25.0,149.1,172.0,182.6,32.4,1,130.0,86.3,1.299783
2022-06-30 21:39:30+00:00,Afternoon Run,6.014199,3682,3685,314.0,Run,7395188303,11,5,0,...,4.392,28.0,154.1,180.0,183.2,48.2,6,102.0,84.4,1.662140
2022-06-30 00:12:17+00:00,Afternoon Golf,6.437597,10586,14280,120.0,Golf,7390944431,0,2,0,...,2.624,28.0,102.5,148.0,93.8,68.4,0,29.0,NaN,0.164368


In [42]:
strava_activities_clean.head(3)

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,max_speed,average_cadence,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-06-30 21:39:30+00:00,Afternoon Run,6.014199,3682,3685,314.0,Run,7395188303,11,3,0,...,4.392,84.4,28.0,154.1,180.0,183.2,48.2,6,102.0,1.662140
2022-06-30 00:12:17+00:00,Afternoon Golf,6.437597,10586,14280,120.0,Golf,7390944431,0,2,0,...,2.624,NaN,28.0,102.5,148.0,93.8,68.4,0,29.0,0.164368
2022-06-29 19:24:38+00:00,Lunch Run,4.054633,2165,2168,132.0,Run,7389543804,3,1,0,...,4.302,87.0,27.0,150.8,167.0,181.4,49.0,0,47.0,1.302540


In [106]:
x = 'start_time'
y = 'average_speed'
df = strava_activities_clean[(strava_activities_clean.sport_type == 'Run') & (strava_activities_clean.distance.between(3.99, 8.5))]

fig = px.scatter(data_frame=df, 
                        y=y,   # choice 
                        x=x,
                        #hover_data=['distance', 'calories'], 
                        #labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title=str(y + ' x ' + x),
                        #size='distance',
                        #color='distance',
                        template='plotly',
                        height=500, 
                        width=800, 
                        #trendline='ols'
                        )

In [107]:
fig.show()

In [76]:
type(strava_activities_clean['start_time'][2])

str

In [66]:
pd_dt_obj

Timestamp('2022-06-28 19:05:39+0000', tz='UTC')

In [65]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

Timestamp('2022-06-28 20:05:39+0000', tz='UTC')

In [67]:
regular_dt_obj

numpy.datetime64('2022-06-28T19:05:39.000000000')